---

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1.1

--2023-10-18 20:17:43--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1.1
Resolving zenodo.org (zenodo.org)... 188.185.33.206, 188.185.10.78, 188.185.22.33, ...
Connecting to zenodo.org (zenodo.org)|188.185.33.206|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2023-10-18 20:17:44--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1.1’

SO_vectors_200.bin? 100%[===================>]   1.35G  30.3MB/s    in 45s     

2023-10-18 20:18:29 (30.7 MB/s) - ‘SO_vectors_200.bin?download=1.1’ saved [1453905423/1453905423]



In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1.1", binary=True)

In [ ]:
wv_embeddings

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [ ]:
print(len(wv_embeddings))

1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [ ]:
from gensim.models import Word2Vec


# Поиск наиболее близких слов к "dog"
similar_words = wv_embeddings.most_similar("dog", topn=5)

# Проверка, входит ли слово "cat" в топ-5 близких слов
cat_in_top_5 = False
for word, similarity in similar_words:
    if word == "cat":
        cat_in_top_5 = True
        break

if cat_in_top_5:
    print("Слово 'cat' входит в топ-5 близких слов к 'dog'")
else:
    print("Слово 'cat' не входит в топ-5 близких слов к 'dog'")

Слово 'cat' не входит в топ-5 близких слов к 'dog'


In [ ]:
# Измерение близости между "cat" и "dog"
similarity = wv_embeddings.similarity('cat', 'dog')

print('Similarity between "cat" and "dog":', similarity)

Similarity between "cat" and "dog": 0.6852341


In [ ]:
from gensim.models import Word2Vec

for i in range(len(wv_embeddings)):

  # Поиск наиболее близких слов к "dog"
  similar_words = wv_embeddings.most_similar("dog", topn=i)

  # Проверка, входит ли слово "cat" в топ близких слов
  cat_in_top = False
  for word, similarity in similar_words:
      if word == "cat":
          cat_in_top = True
          break

  if cat_in_top > 0:
      print(f"Слово 'cat' входит в топ-{i} близких слов к 'dog'")
      break


Слово 'cat' входит в топ-26 близких слов к 'dog'


### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [26]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    # Tokenize the question
    words = tokenizer.tokenize(question)

    # Initialize an empty array to store the word embeddings
    vec = np.zeros(dim)

    # Initialize a counter to keep track of the number of known words
    num_known_words = 0

    # Iterate through the words in the question
    for word in words:
        # Check if the word has a pre-trained embedding
        if word in embeddings:
            # Add the word embedding to the vector
            vec +=embeddings[word]
            num_known_words += 1

    # If there are known words in the question, divide the vector by the number of known words
    if num_known_words > 0:
        vec /= num_known_words

    return vec

    '''your code'''

In [ ]:
question = "What is the biggest city?"
question_vector = question_to_vec(question, wv_embeddings, tokenizer)

print("Векторный представления вопроса:", question_vector)


Векторный представления вопроса: [ 1.39146733 -1.3887133   0.30325219 -0.28350338 -0.44755676  0.30219007
  0.0560236  -0.35133795  1.85625458 -1.78460902 -0.34924635  1.01868618
  0.14429855 -0.70773312 -0.69611089  0.40300557 -1.87097448 -0.98037037
 -1.67754689  0.57440775 -1.01943637 -1.79974431  0.54812799 -0.41991996
 -3.06261897  1.29558037 -1.70816547 -1.49023932  0.74754554 -0.87412395
 -1.04671094 -1.02201992 -0.53654325 -0.41665719  0.06319016  0.24778077
  0.15469557  1.12321788 -2.47506002  0.87982598 -1.35152924  0.96647254
 -1.20061487  0.09736997 -0.05867365 -0.94838068 -0.68032688 -0.0109445
 -0.35472399 -0.68956918  2.1215198  -0.43577182 -4.66424298  0.67638874
 -0.57153475  2.42840058  1.28773066 -0.81739846  1.27874023 -0.05985743
 -0.94890014 -0.85761774 -0.44097778  0.85156637  0.86392459  0.90005356
  0.22255594 -0.51939642  0.25713938  0.34099245 -0.61098303 -2.75033891
 -0.38471007 -0.522158   -0.74399145  1.11927592 -0.58707865  0.93562651
 -1.95322108 -1.358

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1.1", binary=True)

In [ ]:
# Определяем предложение
sentence = "I love neural networks"

words = tokenizer.tokenize(sentence)

# Разбиваем предложение на слова
#words = sentence.split()

# Вычисляем векторные представления слов
vectors = []

for word in words:
  try:
    vectors.append(wv_embeddings[word])
  except:
    continue

# Определяем третью компоненту векторного представления предложения
third_component = round(vectors[2][2], 2)

print(f"The third component of the sentence vector is: {third_component}")


The third component of the sentence vector is: -0.8199999928474426


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [32]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    '''your code'''
    dup_ranks = np.array(dup_ranks)
    return np.mean((dup_ranks <= k).astype(float))

In [33]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    '''your code'''
    dup_ranks = np.array(dup_ranks)
    return np.mean((1/(np.log2(1 + dup_ranks)))*(dup_ranks <= k).astype(float))

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [candidates_ranking[0].index(copy_answers[0])+1]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
!unzip stackoverflow_similar_questions.zip

unzip:  cannot find or open stackoverflow_similar_questions.zip, stackoverflow_similar_questions.zip.zip or stackoverflow_similar_questions.zip.ZIP.


Считайте данные.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

Нам понадобиться только файл validation.

In [22]:
validation_data = read_corpus('/content/drive/MyDrive/validation.tsv')

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [29]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [24]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    question_vec = question_to_vec(question, embeddings, tokenizer, dim)
    candidate_vecs = [question_to_vec(candidate, embeddings, tokenizer, dim) for candidate in candidates]

    # Вычисляем косинусное расстояние между вопросом и кандидатами
    similarities = cosine_similarity([question_vec], candidate_vecs).flatten()

    # Создаем список пар (позиция, кандидат) и сортируем его по убыванию сходства
    ranked_candidates = [(i, candidate) for i, candidate in enumerate(candidates)]
    ranked_candidates.sort(key=lambda x: -similarities[x[0]])

    return ranked_candidates

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(0, 'Getting all list items of an unordered list in PHP'), #скрыт
            (2, 'select2 not displaying search results'), #скрыт
            (1, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [20]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.285 | Hits@   1: 0.285
DCG@   5: 0.342 | Hits@   5: 0.393
DCG@  10: 0.360 | Hits@  10: 0.449
DCG@ 100: 0.406 | Hits@ 100: 0.679
DCG@ 500: 0.431 | Hits@ 500: 0.879
DCG@1000: 0.444 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [2]:
pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 49.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=d4dda066bed6b319e011d87b25ca0e03e5510eaff3115857f8a5945c407a9554
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [9]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

In [10]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

In [11]:
train_data = read_corpus('/content/drive/MyDrive/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [13]:
nltk.download('stopwords')
stopWords = set(stopwords.words('english'))

def preproc_nltk(text):
    return ' '.join([word for word in word_tokenize(text.lower()) if word not in stopWords])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
nltk.download('punkt')

couple_list = [couple[0] + str(' ') + couple[1] for couple in train_data]
words = [preproc_nltk(question).split() for question in couple_list]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                vector_size=200,                 # embedding vector size
                min_count=9,             # consider words that occured at least 5 times
                window=3).wv

In [18]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

In [30]:
import numpy as np
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [35]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    dcg_score_res = np.mean(dcg_score(wv_ranking, k))
    hits_count_res = np.mean(hits_count(wv_ranking, k))
    print(f"DCG@{k}: {dcg_score_res} | Hits@{k}: {hits_count_res}" )

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@1: 0.237 | Hits@1: 0.237
DCG@5: 0.30163524227722216 | Hits@5: 0.358
DCG@10: 0.3229674786425551 | Hits@10: 0.424
DCG@100: 0.36720759838167055 | Hits@100: 0.641
DCG@500: 0.39493386995158763 | Hits@500: 0.858
DCG@1000: 0.4099744568821218 | Hits@1000: 1.0


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:



1. **Принцип токенизации**: Определение, каким образом текст будет разбит на отдельные токены, зависит от конкретной задачи и текстовых данных. В зависимости от требований, можно использовать более гибкие методы, такие как WordPiece или SentencePiece, для более точной токенизации. Иногда простое разделение текста на слова также может быть приемлемым. Выбор метода токенизации должен быть согласован с характеристиками текстов и целями обработки.

2. **Нормализация слов**: Процесс нормализации слов, включая стемминг и лемматизацию, может значительно улучшить результаты в задачах, связанных с поиском или сопоставлением текстов. Эти методы могут помочь уменьшить разнообразие форм слов, что в свою очередь снизит размерность данных и упростит сравнение между текстами.

3. **Эмбеддинги**: Эффективность эмбеддингов зависит от конкретной задачи и данных, с которыми мы работаем. Например,
<br>
<br>Word2Vec - хорошим выбором для задач классификации текстов, где важен смысл и семантические связи между словами (отзывы на отели, комментарии в соц.сетях), также для поиска синонимов и семантически близких слов
<br>
<br>GloVe - метод для получения векторных представлений слов, основанный на статистике совместной встречаемости слов в больших текстовых корпусах => лучше использовать на структурированных данных (таблицы, документы), где семантика и контекст слов менее важны

4. **Плохое качество решения задачи**: Плохое качество может иметь разные причины, включая недостаток данных, неправильный выбор метода обработки текста, недостаточную глубину модели, проблемы с переобучением и другие факторы.
<br>Для выявления и понимания этих причин часто требуются анализ данных и проведение экспериментов.

5. **Подход к решению задачи**:
Чтобы улучшить качество решения нужно:
- Провести анализ данных
- Изучить особенности данных, выбросы, аномалии
- Подобрать методы токенизации, нормализации
- Подобрать подходящие эмбеддинги и модели
- Проверить, что модель не переобучена